In [ ]:
#분석
import pandas as pd
import numpy as  np
import random as rnd

In [ ]:
# 시각화
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
#라이브러리
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer 
from sklearn.preprocessing import RobustScaler 
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.model_selection import train_test_split

### 데이터 불러오기

- Survival 생존여부 0=No 1=Yes
- pclass 사회-경제적지위 1=1st, 2=2nd, 3=3rd
- sex : 성별
- Age : 나이
- sibsp : 타이타닉호에 탑승한 형제-자매 수
- parch ; 타이타닉호에 탑승한 부모-자녀 수
- ticket : 티켓번호
- fare : 탑승 요금
- cabin : 방번호
- embarked : 탑승 지역(항구 위치)
C = Cherbourg,
  Q = Queenstown,
S = Southampton

### EDA

1. 데이터의 열 별 정보를 확인 (결측치 확인)
2. 변수간 상관관계 확인

In [ ]:
train_df = pd.read_csv('./train.csv')
test_df = pd.read_csv('./test.csv')

In [ ]:
train_df

In [ ]:
# type을 알아보기 위해서 작성.
train_df.info()

In [ ]:
#데이터를 요약해서 보기 위해서 작성.
train_df.describe() 

In [ ]:
#결측값 확인.
print(train_df.isnull().sum())
print(train_df.info())

#### 데이터 상관관계 확인

In [ ]:
#pclass 통한 생존율 확인
train_df.groupby(['Pclass', 'Survived'])['Survived'].count()

In [ ]:
 #pclass 높을 수록 생존율 높음 (62.9% vs 47.3% vs 24.2%)
train_df.groupby('Pclass')['Survived'].mean()

In [ ]:
#성별에 따른 생존 여부
train_df.groupby(['Sex','Survived'])['Survived'].count()

In [ ]:
#pclass 높을 수록 fare(탑승 요금) 높음
train_df.groupby('Pclass')['Fare'].median() 

In [ ]:
#pclass 높을 수록 age 높아짐
train_df.groupby('Pclass')['Age'].median()

In [ ]:
# 상관관계 분석
train_df.corr() 

#### 시각화

In [ ]:
#성별에 따른 생존 여부
sns.barplot(x="Sex", y="Survived", data=train_df)

In [ ]:
#pclass에 따른 생존률
sns.barplot(x='Pclass', y='Survived', data=train_df) 

In [ ]:
# 성별과 선실에 따른 생존여부 확인
sns.barplot(x='Sex', y='Survived', hue='Pclass', data=train_df)

In [ ]:
# 선착지에에 따른 생존여부 확인.
sns.barplot(x='Embarked', y='Survived', data=train_df)
#Southampton이 생존율이 제일 낮은 것을 확인.

In [ ]:
# 나이에 따른 생존률 
grid = sns.FacetGrid(train_df, col='Survived')
grid.map(plt.hist, 'Age', alpha=0.5, bins=20)
grid.add_legend()
plt.show() 
#어린 아이들의 생존률이 높음

In [ ]:
#나이에 따른 pclass 분포 확인
grid = sns.FacetGrid(train_df, col='Pclass')
grid.map(plt.hist, 'Age', alpha=0.5, bins=20)
grid.add_legend()
plt.show() 
# 각 pclass는 나이대 분포가 다양함

In [ ]:
#Survived와 Pclass에 따른 Age 분포 확인 
grid = sns.FacetGrid(train_df, col="Survived", row="Pclass")
grid.map(plt.hist, 'Age', alpha = .5, bins=20)
grid.add_legend(); 
#Pclass=2 에 탄 어린아이들의 survived 확률이 높음

In [ ]:
#칼럼 간 상관관계 시각화
plt.figure(figsize=(8, 8))
sns.heatmap(data=train_df.corr(), annot=True)

### 전처리

#### (1) Age 전처리

In [ ]:
# Title 변수 생성

train_df['Title'] = train_df['Name'].str.extract(' ([A-Za-z]+)\.', expand=False) 
train_df['Title'].value_counts()

In [ ]:
name_mapping = {"Mr": 0, "Miss": 1, "Mrs": 2, "Master": 3, "Dr": 4, "Rev": 4, "Col": 4, "Major": 4, "Mlle": 4,"Countess": 4, "Ms": 4, "Lady": 4, "Jonkheer": 4, "Don": 4, "Dona" : 4, "Mme": 4,"Capt": 4,"Sir": 4 } 

x_data_test = [train_df]
for dataset in x_data_test : 
    dataset['Title'] = dataset['Title'].map(name_mapping) 

train_df.head()

In [ ]:
train_df.groupby(['Title']).mean()

In [ ]:
# Age 결측값에 Title에 따른 평균값 넣음

train_df.loc[(train_df.Age.isnull())&(train_df.Title==0),'Age']=32
train_df.loc[(train_df.Age.isnull())&(train_df.Title==1),'Age']=22
train_df.loc[(train_df.Age.isnull())&(train_df.Title==2),'Age']=36
train_df.loc[(train_df.Age.isnull())&(train_df.Title==3),'Age']=5
train_df.loc[(train_df.Age.isnull())&(train_df.Title==4),'Age']=30

#### (2) Sex 전처리

In [ ]:
# male 값 : 0 female 값: 1로 설정 
train_df['Sex'] = train_df['Sex'].map({'male':0,
                                       'female':1})

train_df

#### (3) Embarked 전처리

In [ ]:
# Southampton:0, Cherbourg:1, Queenstown:2
train_df['Embarked'] = train_df['Embarked'].map({'S':0,
                                                'C':1,
                                                'Q':2})

train_df

In [ ]:
#Embarked 결측치 2개니까 삭제 or 중앙값 대체.
#train_df = train_df.dropna()
train_df['Embarked'].fillna(train_df["Embarked"].median(), inplace=True)

In [ ]:
#유니크 값 확인.
train_df['Embarked'].unique()

In [ ]:
# fare 4 구간으로 나누기, 연속형 변수 였던 fare을 범주형 변수로 전환
train_df['FareBand'] = pd.qcut(train_df['Fare'], 4, labels = [1, 2, 3, 4])
test_df['Fare'].fillna(test_df["Fare"].median(), inplace = True)
test_df['FareBand'] = pd.qcut(test_df['Fare'], 4, labels = [1, 2, 3, 4])

In [ ]:
train_df['FareBand'] = train_df['FareBand'].astype(int)
test_df['FareBand'] = test_df['FareBand'].astype(int)

In [ ]:
test_df

In [ ]:
# Cabin, Ticket, Name 드랍

#### 결측값 확인

In [ ]:
print(train_df.isnull().sum())

##### 모델링 전 확인

In [ ]:
train_df.columns

In [ ]:
sds = StandardScaler()
numeric_columns = ['Pclass', 'Age', 'Parch', 'Fare','Sex','Embarked', 'SibSp']

train_df_norm = sds.fit_transform(train_df[numeric_columns])

In [ ]:
train_df_norm.shape

### 모델링

1. Logistic Regression (로지스틱 회귀)
2. Decision Tree (의사결정 나무)
3. Random Forest (랜덤포레스트)
4. XGBoost (XGBoost)

In [ ]:
model_lr = LogisticRegression()
model_dt = DecisionTreeClassifier(criterion = 'entropy', max_depth = 1, random_state = 0)
model_rf = RandomForestClassifier(n_estimators = 8, random_state = 0)
model_xgb = xgb.XGBClassifier()

### 학습
1. 학습데이터에서 독립변수와 종속변수와를 구분하고 모델을 학습.

In [ ]:
train_x, valid_x, train_y, valid_y = train_test_split(train_df[numeric_columns], train_df['Survived'], 
                                                        test_size=0.2, shuffle=True)

In [ ]:
model_lr.fit(train_x, train_y)
model_dt.fit(train_x, train_y)
model_rf.fit(train_x, train_y)
model_xgb.fit(train_x, train_y)

In [ ]:
#Linear Regression
print('train score : {:.3f}'.format(model_lr.score(train_x, train_y)))
print('test socre : {:.3f}'.format(model_lr.score(valid_x, valid_y)))

In [ ]:
#Decision Ttree
print('train score : {:.3f}'.format(model_dt.score(train_x, train_y)))
print('test socre : {:.3f}'.format(model_dt.score(valid_x, valid_y)))

In [ ]:
#Random Forest
print('train score : {:.3f}'.format(model_rf.score(train_x, train_y)))
print('test socre : {:.3f}'.format(model_rf.score(valid_x, valid_y)))

In [ ]:
#XGB
print('train score : {:.3f}'.format(model_xgb.score(train_x, train_y)))
print('test socre : {:.3f}'.format(model_xgb.score(valid_x, valid_y)))

### 테스트

In [ ]:
test_df = pd.read_csv(f"./test.csv", index_col=0)

print(test_df.isnull().sum())
print(test_df.info())

In [ ]:
# Title 변수 생성

test_df['Title'] = test_df['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)

name_mapping = {"Mr": 0, "Miss": 1, "Mrs": 2, "Master": 3, "Dr": 4, "Rev": 4, "Col": 4, "Major": 4, "Mlle": 4,"Countess": 4, "Ms": 4, "Lady": 4, "Jonkheer": 4, "Don": 4, "Dona" : 4, "Mme": 4,"Capt": 4,"Sir": 4 } 

x_data_test = [test_df]
for dataset in x_data_test : 
    dataset['Title'] = dataset['Title'].map(name_mapping) 

# Age 결측값에 Title에 따른 평균값 넣음

test_df.loc[(test_df.Age.isnull())&(test_df.Title==0),'Age']=32
test_df.loc[(test_df.Age.isnull())&(test_df.Title==1),'Age']=22
test_df.loc[(test_df.Age.isnull())&(test_df.Title==2),'Age']=36
test_df.loc[(test_df.Age.isnull())&(test_df.Title==3),'Age']=5
test_df.loc[(test_df.Age.isnull())&(test_df.Title==4),'Age']=30

test_df['Sex'] = test_df['Sex'].map({'male':0, 'female':1})
test_df['Embarked'] = test_df['Embarked'].map({'S':0, 'C':1, 'Q':2})
test_df['Fare'].fillna(test_df["Fare"].median(), inplace=True)

In [ ]:
test_df

In [ ]:
y_pred = model_lr.predict(test_df[numeric_columns])

In [ ]:
y_pred

In [ ]:
df_submission = pd.read_csv(f"./submission.csv")
df_submission['Survived'] = y_pred
df_submission.to_csv('submission_1.csv', index=False)